## 1. 管理工具安装
### 1.1 langchain 安装

In [1]:
!pip install langchain

# conda 
# conda install -c conda-forge langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


### 1.2 其他库安装

In [2]:
# 直接转换为 REST API
!pip install "langserve[all]"

# 观测平台
!pip install -U langsmith

# LangGraph
!pip install -U langgraph

# DeepSeek
!pip install langchain-deepseek

# 外部资源集成
!pip install langchain_community

# dotenv管理密钥，加载环境变量等
!pip install python-dotenv

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.6 MB/s  0:00:0036m-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [langserve]/7 [fastapi]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.2/603.2 kB 9.1 MB/s  0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.4.37
    Uninstalling langsmith-0.4.37:
      Successfully uninstalled langsmith-0.4.37
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [langsmith]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Attempting uninstall: langgraph-prebuilt
    Found existing installation: langgraph-prebuilt 1.0.1
    Uninstalling langgraph-prebuilt


### 1.3 常见的处理链包含三个要素

1. 语言模型：核心推理引擎
2. 提示词模板：提供指令
3. 输出解释器：转换为易于使用的格式，便于下游处理

#### 1.3.1 语言模型

langchain语言模型主要分为两种：

1. LLM 通用模型：接收字符串作为输入，输出字符串作为输出。
2. ChatModel 对话模型：接收消息列表，输出“消息”，用于一问一答。

消息：由 BaseMessage 类定义，包含文本、时间戳、用户标识符等属性。主要的有

1. 消息的内容：文本，通常是字符串
2. 角色：消息的发送方

角色： LangChain 用于区分不同角色的对象

1. HumanMessage：人类（用户）输入的BaseMessage。
2. AIMessage：AI助手（大模型）输出的BaseMessage。
3. SystemMessage：系统预设的BaseMessage。
4. FunctionMessage：自定义函数输出的BaseMessage。
5. ToolMessage：调用第三方工具输出的BaseMessage。
6. ChatMessage：自定义角色。

方法： LCEL 默认实现同步调用方法，最常见的是 invoke 方法，接受一个 BaseMessage 对象作为参数，并返回一个 BaseMessage 对象作为结果。

比如：
LLMs.invoke：输入输出都是字符串
ChatModels.invoke：输入输出都是BaseMessage对象

In [6]:
!pip install dashscope
from langchain_community.llms.tongyi import Tongyi
from dotenv import load_dotenv

load_dotenv()

llm = Tongyi()

print(llm.invoke("你好"))

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
你好呀！✨ 很高兴见到你！今天过得怎么样呀？希望你度过了愉快的一天。我随时准备好陪你聊天、帮你解决问题，或者就这样轻松愉快地闲聊一会儿。有什么想跟我分享的吗？ 🌟


In [8]:
from langchain_deepseek import ChatDeepSeek
from dotenv import load_dotenv
load_dotenv()


deepseek = ChatDeepSeek(model="deepseek-chat")
print(deepseek.invoke("你好"))

content='你好！很高兴见到你！😊 我是DeepSeek，由深度求索公司创造的AI助手。无论你有什么问题、需要什么帮助，或者只是想聊聊天，我都很乐意为你提供支持！\n\n我可以帮你解答各种问题，处理文档，进行创作和分析等等。有什么我可以为你做的吗？我会尽我所能热情地帮助你！✨' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 5, 'total_tokens': 77, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 5}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': 'c092fdbb-5087-4f18-9cad-61e7747fd2d5', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--3a99dee5-0fbd-4e7f-89d7-923b9bd36d2d-0' usage_metadata={'input_tokens': 5, 'output_tokens': 72, 'total_tokens': 77, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}


#### 1.3.2 提示词模板

提示词模板是用于生成提示词的模板。例如，`{name}，你好！我是你的朋友小爱同学。`就是一个提示词模板。

提示词模板中可以使用变量，变量名由字母、数字和下划线组成，且必须以字母开头。例如，`{name}，你好！我是你的朋友小爱同学。`中的`{name}`就是一个变量。

In [12]:
from langchain_core.prompts  import PromptTemplate

prompt = PromptTemplate.from_template(
    "Hello, I am a {model_name}. How can I help you today?"
)

print(prompt.format(model_name="language model"))
print(prompt.invoke({"model_name": "language model"}))

Hello, I am a language model. How can I help you today?
text='Hello, I am a language model. How can I help you today?'


In [16]:
# 使用 ChatPromptTemplate 创建对话模板
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI assistant named {assistant_name}."),
        ("human", "Hello, my name is {user_name}. {question}")
    ]
)

messages = chat_prompt.format_messages(
    assistant_name="ChatGPT",
    user_name="Alice",
    question="Can you tell me a joke?",
)
for message in messages:
    print(f"{message.type}: {message.content}")

system: You are a helpful AI assistant named ChatGPT.
human: Hello, my name is Alice. Can you tell me a joke?


In [17]:
from langchain_core.prompts import PromptTemplate

complex_prompt= PromptTemplate(
    input_variables=["topic", "audience", "tone"],
    template="""
    请为{audience}写一篇关于{topic}的文章。
    写作风格应该是{tone}的。
    
    文章要求：
    - 内容准确且有用
    - 结构清晰
    - 适合目标受众
    """
)

formtted_prompt = complex_prompt.format(
    topic="人工智能的未来",
    audience="大学生",
    tone="幽默风趣"
)

print(formtted_prompt)


    请为大学生写一篇关于人工智能的未来的文章。
    写作风格应该是幽默风趣的。

    文章要求：
    - 内容准确且有用
    - 结构清晰
    - 适合目标受众
    


In [18]:
from langchain_core.prompts import PromptTemplate
# f-string 风格、jinja2语法 ...
f_string_prompt = PromptTemplate.from_template(
    "分析以下{data_type}数据：\n{data}\n\n请提供{analysis_type}分析。"
)
formatted_prompt = f_string_prompt.format(
    data_type="销售",
    data="2023年第一季度销售额为100万元，第二季度为150万元。",
    analysis_type="趋势"
)
print(formatted_prompt)

分析以下销售数据：
2023年第一季度销售额为100万元，第二季度为150万元。

请提供趋势分析。


In [21]:
from langchain_core.prompts import PromptTemplate

# 带条件逻辑的模板
conditional_prompt = PromptTemplate(
    input_variables=["user_type", "question"],
    template_format="jinja2", # 确保使用 Jinja2 引擎
    template="""
    {%- if user_type == "expert" -%}
    作为专家，请详细回答：{{ question }}  
    {%- else -%}
    请用简单易懂的方式回答：{{ question }} 
    {%- endif -%}
    """
)

formatted_prompt = conditional_prompt.format(
    user_type="expert",
    question="量子计算的基本原理是什么？"
)
print(formatted_prompt)

作为专家，请详细回答：量子计算的基本原理是什么？


#### 1.3.3 输出解析器

输出解析器是 LangChain 中的重要组件，用于将大语言模型(LLM)的原始文本输出转换为结构化数据格式，便于程序进一步处理和使用。

主要功能
- 格式化输出: 将 LLM 的自然语言输出解析为 JSON、XML、列表等结构化格式
- 类型转换: 将文本数据转换为 Python 对象（字典、列表、自定义类等）
- 数据验证: 确保输出符合预期的格式和约束条件
- 错误处理: 处理解析失败的情况，提供重试机制

常见的输出解析器类型


In [ ]:
# JSON 输出解析器
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class PersonInfo(BaseModel):
    name: str = Field(..., description="人物姓名")
    age: int = Field(..., description="年龄")
    occupation: str = Field(..., description="职业")
    
parser = PydanticOutputParser(pydantic_object=PersonInfo)

from langchain_core.output_parsers import CommaSeparatedListOutputParser
list_parser = CommaSeparatedListOutputParser()
# 将 "苹果,香蕉,橙子" 解析为 ["苹果", "香蕉", "橙子"]

# 结构化输出解析器
from langchain_core.output_parsers import StructuredOutputParser, ResponseSchema
response_schemas = [
    ResponseSchema(name="answer", description="问题的答案"),
    ResponseSchema(name="source", description="答案来源")
]
structured_parser = StructuredOutputParser.from_response_schemas(response_schemas)


In [22]:
# 手动使用解析器
from langchain_core.output_parsers import CommaSeparatedListOutputParser

list_parser = CommaSeparatedListOutputParser()
# 模拟 LLM 的原始输出
raw_output = "Python, Java, JavaScript, C++, Go"

parser_result = list_parser.parse(raw_output)
print("手动解析结果:", parser_result)


手动解析结果: ['Python', 'Java', 'JavaScript', 'C++', 'Go']


In [26]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_community.llms.tongyi import Tongyi

list_parser = CommaSeparatedListOutputParser()
llm = Tongyi()

def simple_list_generator(category: str):
    prompt = f"""
    请列出5个{category}的例子。{list_parser.get_format_instructions()}"""
    
    print(prompt)
    
    # 直接调用LLM
    response = llm.invoke(prompt)
    
    # 解析结果
    return list_parser.parse(response)

# 使用
fruits = simple_list_generator("水果")
print("水果列表:", fruits)

languages = simple_list_generator("编程语言")
print("编程语言列表:", languages)


    请列出5个水果的例子。Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
水果列表: ['苹果', '香蕉', '橙子', '葡萄', '草莓']

    请列出5个编程语言的例子。Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
编程语言列表: ['Python', 'Java', 'JavaScript', 'C++', 'Ruby']


In [27]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.llms.tongyi import Tongyi

# 1. 创建输出解析器
parser = CommaSeparatedListOutputParser()
# 2. 获取格式化指令
format_instructions = parser.get_format_instructions()
print("格式化指令:", format_instructions)

# 3. 创建包含格式指令的提示模板
prompt = PromptTemplate(
    template="请列出5个{category}的例子。\n{format_instructions}",
    input_variables=["category"],
    partial_variables={"format_instructions": format_instructions}
)


# 4. 初始化 LLM
llm = Tongyi(temperature=0)

# 5. 创建完整的链
chain = prompt | llm | parser
# 6. 运行链
try:
    results = chain.invoke({"category": "水果"})
    print("解析结果:", results)
    print("类型:", type(results))
    
    for i, res in enumerate(results, 1):
        print(f"{i}. {res}")
except Exception as e:
    print("链运行出错:", e)
    
# 7. 更复杂的示例 - 多个类别
categories = ["编程语言", "运动项目", "乐器"]

for category in categories:
    print(f"\n=== {category} ===")
    try:
        result = chain.invoke({"category": category})
        for item in result:
            print(f"• {item}")
    except Exception as e:
        print(f"处理 {category} 时出错: {e}")

格式化指令: Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
解析结果: ['苹果', '香蕉', '橙子', '葡萄', '草莓']
类型: <class 'list'>
1. 苹果
2. 香蕉
3. 橙子
4. 葡萄
5. 草莓

=== 编程语言 ===
• Python
• Java
• C++
• JavaScript
• Ruby

=== 运动项目 ===
• 足球
• 篮球
• 游泳
• 田径
• 网球

=== 乐器 ===
• 钢琴
• 吉他
• 小提琴
• 鼓
• 长笛


In [28]:
# 购物清单
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Tongyi

# 1. 初始化组件
parser = CommaSeparatedListOutputParser()
llm = Tongyi(temperature=0)

# 2. 创建提示模板
shopping_prompt = PromptTemplate(
    template="根据{meal_type}，生成一个包含5个食材的购物清单。\n{format_instructions}",
    input_variables=["meal_type"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# 3. 创建链
shopping_chain = shopping_prompt | llm | parser

# 4. 生成不同餐型的购物清单
meals = ["早餐", "午餐", "晚餐"]
shopping_lists = {}

for meal in meals:
    shopping_lists[meal] = shopping_chain.invoke({"meal_type": meal})
    print(f"{meal}购物清单: {shopping_lists[meal]}")

# 5. 格式化输出
print("\n=== 详细购物清单 ===")
for meal, items in shopping_lists.items():
    print(f"\n🍽️ {meal}:")
    for i, item in enumerate(items, 1):
        print(f"  {i}. {item}")

早餐购物清单: ['面包', '鸡蛋', '牛奶', '黄油', '香蕉']
午餐购物清单: ['鸡胸肉', '生菜', '番茄', '全麦面包', '酸奶']
晚餐购物清单: ['鸡胸肉', '西兰花', '大米', '橄榄油', '大蒜']

=== 详细购物清单 ===

🍽️ 早餐:
  1. 面包
  2. 鸡蛋
  3. 牛奶
  4. 黄油
  5. 香蕉

🍽️ 午餐:
  1. 鸡胸肉
  2. 生菜
  3. 番茄
  4. 全麦面包
  5. 酸奶

🍽️ 晚餐:
  1. 鸡胸肉
  2. 西兰花
  3. 大米
  4. 橄榄油
  5. 大蒜


In [29]:
# 多场景
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import Tongyi

parser = CommaSeparatedListOutputParser()
llm = Tongyi(temperature=0)

list_chain = (
    PromptTemplate(
        template="请列出5个{category}的{item_type}。\n{format_instructions}",
        input_variables=["category", "item_type"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    | llm
    | parser
)

# 多场景使用
scenarios = [
    {"category": "早餐", "item_type": "食材"},
    {"category": "办公室", "item_type": "用品"},
    {"category": "旅行", "item_type": "必需品"},
    {"category": "健身", "item_type": "器材"},
    {"category": "学习", "item_type": "工具"}
]

for scenario in scenarios:
    result = list_chain.invoke(scenario)
    print(f"\n{scenario['category']}的{scenario['item_type']}:")
    for i, item in enumerate(result, 1):
        print(f"  {i}. {item}")


早餐的食材:
  1. 鸡蛋
  2. 牛奶
  3. 面包
  4. 燕麦
  5. 香蕉

办公室的用品:
  1. 笔
  2. 笔记本
  3. 订书机
  4. 文件夹
  5. 计算器

旅行的必需品:
  1. 护照
  2. 机票
  3. 钱包
  4. 手机
  5. 充电器

健身的器材:
  1. 跑步机
  2. 哑铃
  3. 杠铃
  4. 动感单车
  5. 仰卧板

学习的工具:
  1. Anki
  2. Quizlet
  3. Khan Academy
  4. Coursera
  5. Notion


#### 1.3.4 使用 LCEL 的好处

1. 简洁语法: 使用管道操作符 | 创建链，代码直观易读
2. 自动类型推断: 组件间数据类型自动转换，减少手动处理
3. 并行处理: 内置并行执行能力，提高处理效率
4. 流式处理: 原生支持流式输出，适合实时应用
5. 智能缓存: 自动缓存中间结果，避免重复计算
6. 内置调试: 更好的调试和监控能力
7. 插件化: 支持自定义组件和扩展
8. 标准接口: 统一的 Runnable 接口
9. 版本管理: 支持链的版本控制和管理